In [1]:
import math
import numpy as np
import scipy
import scipy.optimize
import matplotlib.pyplot as plt
import csv
#from scipy.optimize import minimize
#import tensorflow as tf

PointConfiguration.set_engine('internal')
#PointConfiguration.set_engine('topcom')
print 'Done.'

Done.


In [2]:
#Helper functions to initialize input data

def reflex_poly(n):
    
    polytopes_list = [1, 5, 6, 7, 8, 25, 26, 27, 28, 29, 30, 31, 82, 83, 84, 85, 219, 220]
    
    P = list(ReflexivePolytope(3, polytopes_list[n-1]-1).vertices())
    pts = []
    
    for j in range(len(P)):
        pts.append(list(P[j]))
    
    return pts

def reflex_poly_list(num_poly):
    
    if num_poly > 18:
        print 'Please input integers <= 18'
        return -1
    
    polytopes_list = [1, 5, 6, 7, 8, 25, 26, 27, 28, 29, 30, 31, 82, 83, 84, 85, 219, 220]

    pts = []
    
    for i in range(num_poly):
        pts.append(reflex_poly(i))

    print 'Initialize reflexive polytope done.'
    
    return pts

def input_data(input_path):
    
    #Code to load 'poly_out.txt'
    with open(input_path) as f:
        pts = []
        pts_tmp = []
        
        for line in f:
            #print line
            pts_tmp = eval(line)
            pts.append(pts_tmp)
        
        '''
        #Code to load 'result.txt'
        for line in f:
            pt = line.split(' ')
            print line
            if line == '\n' or pt == '\n':
                pts.append(pts_tmp)
                pts_tmp = []
                continue
            for i in range(len(pt)):
                print pt[i]
                pt[i] = float(pt[i].strip(','))
            pts_tmp.append(pt)
        if line != '\n':
            pts.append(pts_tmp)
        '''
    
    return pts

print 'Done.'

Done.


In [9]:
#Helper functions to find the Hilbert Series

def exist(pts, latt):
    latt = np.array(latt)
    for i in range(pts.shape[0]):
        if pts[i][0]==latt[0]:
            if pts[i][1]==latt[1]:
                if pts[i][2]==latt[2]:
                    return 1
    return 0

#Compute cross product of three 4-vectors
def four_cross(v1, v2, v3, v4):
    #print "input vectors: ", v1, v2, v3, v4
    v = np.zeros((4,))
    '''
    v1 = list(map(int, v1))
    v2 = list(map(int, v2))
    v3 = list(map(int, v3))
    '''
    counter = 0
    
    for i in range(4):
        mat = [v1[np.arange(len(v1))!=i].tolist(), v2[np.arange(len(v2))!=i].tolist(), v3[np.arange(len(v3))!=i].tolist()]
        mat = matrix(ZZ, mat)
        #print 'matrix: '
        #print mat
        if counter == 1:
            v[i] = -1*mat.det()
            counter = 0
            #print 'neg: ', v[i]
            continue
        elif counter == 0:
            v[i] = mat.det()
            counter = 1
            #print 'pos: ', v[i]
    #print v
    mat = matrix(RR, [v1.tolist(), v2.tolist(), v3.tolist(), v4.tolist()])
    
    if mat.det() < 0:
        #print 'original: ', v
        v = -1*v
        #print 'changed: ', v
    #print 'vector: ', v
    return v

'''
def Hilb(tri, p, output):
    num_tri = len(tri)
    len_tri = len(tri[0])
    triang_list = np.zeros((num_tri, len_tri, 4))
    
    #Convert each element of p into a 4-vector
    #whose last entry equals to 1
    for i in range(num_tri):
        for j in range(len_tri):
            triang_list[i][j] = np.append(np.array(p[tri[i][j]]) , 1)
            
    #print 'triang_list: '
    #print triang_list   '''
def Hilb(triang_list, output):
    triang = np.array(triang_list)
    power = np.zeros(shape = triang.shape)
    Hilb = 0
    t = var('t')
    t1 = var('t1')
    t2 = var('t2')
    t3 = var('t3')
    t4 = var('t4')
    for tri in range(triang.shape[0]):
        hilb = 1
        t_prod = 1
        for i in range(4):
            #Multiplying by -1 is optional
            power[tri][i] = -1*four_cross(triang[tri][i], triang[tri][np.remainder(i+1, 4)], triang[tri][np.remainder(i+2, 4)], triang[tri][np.remainder(i+3, 4)])
            t_prod = t1^(int(power[tri][i][0]))*t2^(int(power[tri][i][1]))*t3^(int(power[tri][i][2]))*t4^int((power[tri][i][3]))
            hilb *= (1-t_prod)^(-1)
        #print 'Hilbert: ', hilb
        Hilb += hilb
    #print 'Hilb: ', Hilb()
    #print Hilb(t1=t, t2=t, t3=t).series(t4, 3)
    #print "p-q web: ", power 
    
    
    m = var('m')
    b1 = var('b1')
    b2 = var('b2')
    b3 = var('b3')
    b4 = var('b4')
    Hilb *= m^4
    
    #print 'Hilb: ', str(Hilb(t1 = (m*b1).exp(), t2 = (m*b2).exp(), t3 = (m*b3).exp(), t4 = (m*4).exp())).replace('e', 'E')
    
    
    Series = Hilb(t1 = (m*b1).exp(), t2 = (m*b2).exp(), t3 = (m*b3).exp(), t4 = (m*4).exp()).series(m==0, 1)
    Series = Series.truncate()
    #Series = limit(Hilb(t1 = (m*b1).exp(), t2 = (m*b2).exp(), t3 = (m*b3).exp(), t4 = (m*4).exp()), m=0)
    #print 'Series: ', Series
    
    if output != 0:
        output.write("%s\n" % Series)

    return Series

def Triang(p, output):
    #Input output = 0 if do not want output
    pts = np.array(p)
    poly = Polyhedron(p)
    pts_max = int(max(np.amax(np.absolute(pts), axis=0)))+1
    pts_new = pts
    for i in range(-pts_max, pts_max):
        for j in range(-pts_max, pts_max):
            for k in range(-pts_max, pts_max):
                latt = [i,j,k]
                if exist(pts, latt)==1:
                    continue
                if poly.contains(latt) == 1:
                    pts_new = np.append(pts_new, np.array(latt).reshape((1,3)), axis = 0)  
    #print 'pts_new: ', pts_new
    pts_new = pts_new.tolist()
    points = PointConfiguration(pts_new)
    triang = points.triangulate()
    triang = [list(triang[i]) for i in range(len(triang))]
    triang_new = []
    check_triang(triang, pts_new, triang_new)
    print 'triangulate: ', triang_new
    
    #Calculate the Hilbert series
    #Series = Hilb(triang, pts_new, output)
    Series = Hilb(triang_new, output)
    
    return Series
'''
def check_triang(triang, pts):
    triang_new = triang
    for i in range(len(triang)):
        #triang_new.append(list(triang[i]))
        poly_tmp = Polyhedron(list(triang[i]))
        for j in range(len(pts)):
            if poly_tmp.contains(pts[j]):
                triang_tmp = PointConfiguration(list(triang[i])).triangulate()
                triang_new.remove(triang[i])
                triang_new.append(triang_tmp)
    return triang_new

'''
def check_one_triang(triang, pts):
    triang_list = triang
    #print 'pts: ', pts
    #print 'triang: ', triang
    vert = [pts[k] for k in triang]
    poly = Polyhedron(vert)
    
    for h in range(len(pts)):
        if h in triang:
            continue
        elif poly.contains(pts[h]):
            triang_list.append(h)
    
    return triang_list

def check_triang(triang, pts, triang_new):
    #triang_new = []
    for i in range(len(triang)):
        check = check_one_triang(triang[i], pts)
        #print "check ", i, ": ", check
        len_check = len(check)
        
        if len_check <= 4:
            triang_save = [np.append(np.array(pts[k]),1).tolist() for k in check]
            #print 'add: ', triang_save
            triang_new.append(triang_save)
            
        else:
            pts_tmp = [pts[j] for j in check]
            #print "vertices: ", pts_tmp
            triang_tmp = PointConfiguration(pts_tmp).triangulate()
            triang_tmp = [list(triang_tmp[i]) for i in range(len(triang_tmp))]
            #print "further triang: ", triang_tmp
            check_triang(triang_tmp, pts_tmp, triang_new)
    #print triang_new


print 'Done.'

Done.


In [7]:
#Helper functions to find the minimum of the volume function
#(None of these work as well as Mathematica yet.)

from scipy.optimize import fsolve
from sympy import nsolve

def min_function(Series, p):
    try:
        return (Series(b1=p[0], b2=p[1], b3=p[2]))^2
    except:
        return -1

def min_constraint(Series, a, b, c):
    if min_function(Series, [a,b,c])== -1:
        print 'bad try: ', a,' ', b , ' ', c 
        return -100
    try:
        print 'val: ', Series(b1=a, b2=b, b3=c)
        return 1-Series(b1=a, b2=b, b3=c)
    except:
        return -100

def Find_Minimum(Series):
    function = lambda p: min_function(Series, p)
    
    constraint = ({'type': 'ineq', 'fun': lambda p:  min_constraint(Series, p[0], p[1], p[2])})
    #constraint = ({'type': 'ineq', 'fun': lambda p:  1-Series(b1=p[0], b2=p[1], b3=p[2])})
    
    solution = scipy.optimize.minimize(function, (0.7,0.2,0.3), constraints=constraint)
    
    return solution

def eval_sol(Series, sol):
    try:
        return Series(b1 = sol[0].rhs(), b2 = sol[1].rhs(), b3 = sol[2].rhs())
    except:
        print 'no sol'
        return CDF(I)

def D_Minimum(Series):
    d1 = diff(Series, b1)
    d2 = diff(Series, b2)
    d3 = diff(Series, b3)
    
    solution = solve([d1 == 0, d2 == 0, d3 == 0], b1, b2, b3)
    #print 'solution: ', solution
    
    sol_len = len(solution)
    
    new_sol = []
    
    for j in range(sol_len):
        if solution[j][0].rhs() in RR and solution[j][1].rhs() in RR and solution[j][2].rhs() in RR:
            vol = eval_sol(Series, solution[j])
            print vol
            if vol not in RR:
                continue
            new_sol.append(vol)
            #new_sol.append(solution[j])
    #solution = new_sol
    
    print 'new_sol: ', new_sol
    
    vol_min_abs = min(new_sol)
    
    return vol_min_abs

def func(p, *d):
    f1, f2, f3 = d
    return (f1(b1 = p[0], b2 = p[1], b3 = p[2]), f2(b1 = p[0], b2 = p[1], b3 = p[2]), f3(b1 = p[0], b2 = p[1], b3 = p[2]))

def constraint(Series, sol):
    vol = Series(b1 = sol[0], b2 = sol[1], b3 = sol[2])
    if vol <= 1 and vol >= 0:
        return 1, vol
    
    return 0, -1

def NSolve(Series):
    d1 = diff(Series, b1)
    d2 = diff(Series, b2)
    d3 = diff(Series, b3)
    d = (d1, d2, d3)
    const = 0
    count = 0
    
    while const == 0:
        d1_0 = np.random.uniform(low=-1, high=1)
        d2_0 = np.random.uniform(low=-1, high=1)
        d3_0 = np.random.uniform(low=-1, high=1)
        print 'reset starting point: ', d1_0, d2_0, d3_0
        try:
            sol = fsolve(func, x0 = np.array([d1_0, d2_0, d3_0]), args = d)
            print 'solution: ', sol
        except:
            continue
        if abs(sol[0]) > 10 or abs(sol[1]) > 10 or abs(sol[2]) > 10:
            continue
        const, vol = constraint(Series, sol)
        
        count += 1
        if count > 1000:
            print 'Infinite loop. Force stop.'
            return -1
    
    print 'Done.'
    
    return vol

print 'Done.'

Done.


In [ ]:
#Generate hilbert series and volume

def generate_hilbert_vol(output_hilb, output_vol):
    input_path = 'output/polygon/poly_out_2.txt'
    pts = input_data(input_path)
    for i in range(len(pts)):
        pts_new = pts[i]
        points = PointConfiguration(pts_new)
        
        #Triangulate
        triang = points.triangulate()
        triang = [list(triang[i]) for i in range(len(triang))]
        triang_new = []
        check_triang(triang, pts_new, triang_new)
        
        #Calculate Hilbert Series (write to output)
        Series = Hilb(triang_new, output_hilb)
        
        #Calculate Volume (write to output)
        vol = NSolve(Series)
        print i,'-th volume: ', vol
        output_vol.write("%s\n" % vol)

output_hilb_path = 'output/series/output_2.txt'
output_vol_path = 'output/vol/output_cube_30.txt'
output_hilb = open(output_hilb_path, 'w')
output_vol = open(output_vol_path, 'w')
generate_hilbert_vol(output_hilb, output_vol)
output_hilb.close()
output_vol.close()
print 'Done.'

/home/carnd/Downloads/SageMath/local/lib/python2.7/site-packages/scipy/optimize/minpack.py:161: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)
/home/carnd/Downloads/SageMath/local/lib/python2.7/site-packages/scipy/optimize/minpack.py:161: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/home/carnd/Downloads/SageMath/local/lib/python2.7/site-packages/scipy/optimize/minpack.py:161: RuntimeWarning: The number of calls to function has reached maxfev = 800.
  warnings.warn(msg, RuntimeWarning)


In [45]:
#Below are all test scripts (optional)

p1 = [[4,0,0],[0,0,0],[0,1,0],[0,0,1]]
#p1 = [[3,0,0],[1,0,0],[0,1,0],[0,0,1]]
#p1 = [[2,0,0],[0,2,0],[0,0,2], [0,0,0]]
#series = Triang(p1, 0)
#print series

pts = [[4, 0, 0],
 [0, 0, 0],
 [0, 1, 0],
 [0, 0, 1],
 [1, 0, 0],
 [2, 0, 0],
 [3, 0, 0]]

points = PointConfiguration(pts)
triang = points.triangulate()
triang = list(triang)
#triang_new = [list(triang[i]) for i in range(len(triang))]
for i in range(len(triang)):
    triang_new.append(list(triang[i]))
triang = triang_new
print triang


[[0, 2, 3, 6], [1, 2, 3, 4], [2, 3, 4, 6]]


In [72]:
triang_new = []
check_triang(triang, pts, triang_new)

print 'result: ', triang_new


check  0 :  [0, 2, 3, 6]
add:  [[4, 0, 0], [0, 1, 0], [0, 0, 1], [3, 0, 0]]
check  1 :  [1, 2, 3, 4]
add:  [[0, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0]]
check  2 :  [2, 3, 4, 6, 5]
vertices:  [[0, 1, 0], [0, 0, 1], [1, 0, 0], [3, 0, 0], [2, 0, 0]]
further triang:  [[0, 1, 2, 4], [0, 1, 3, 4]]
check  0 :  [0, 1, 2, 4]
add:  [[0, 1, 0], [0, 0, 1], [1, 0, 0], [2, 0, 0]]
check  1 :  [0, 1, 3, 4]
add:  [[0, 1, 0], [0, 0, 1], [3, 0, 0], [2, 0, 0]]
result:  [[[4, 0, 0], [0, 1, 0], [0, 0, 1], [3, 0, 0]], [[0, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0]], [[0, 1, 0], [0, 0, 1], [1, 0, 0], [2, 0, 0]], [[0, 1, 0], [0, 0, 1], [3, 0, 0], [2, 0, 0]]]


In [10]:
#p1 = [[4,0,0],[0,0,0],[0,1,0],[0,0,1]]
p1 = [[3,0,0],[0,0,0],[0,1,0],[0,0,1],[1,1,0]]
p2 = [[2,0,0],[0,0,0],[0,1,0],[0,0,1],[1,1,0],[1,0,1]]

p = [p1, p2]

for i in range(len(p)):
    series = Triang(p[i], 0)

    print series

    vol = NSolve(series)
    vol = round(vol, 5)
    print vol

triangulate:  [[[3, 0, 0, 1], [0, 0, 1, 1], [1, 1, 0, 1], [2, 0, 0, 1]], [[0, 0, 0, 1], [0, 1, 0, 1], [0, 0, 1, 1], [1, 1, 0, 1]], [[0, 0, 0, 1], [0, 0, 1, 1], [1, 1, 0, 1], [1, 0, 0, 1]], [[0, 0, 1, 1], [1, 1, 0, 1], [2, 0, 0, 1], [1, 0, 0, 1]]]
1/((b1 - b2)*b1*(b2 + b3 - 4)*b3) - 1/((b1 + 2*b2 + 3*b3 - 12)*(b1 + b2 + 2*b3 - 8)*b2*b3) - 1/((b1 + b2 + 2*b3 - 8)*(b1 + b3 - 4)*b2*b3) - 1/((b1 - b2)*(b1 + b3 - 4)*b2*b3)
reset starting point:  0.858826821695 0.986029015939 0.0511599489556
solution:  [ 3.32287566  1.17712434  1.        ]
Done.
0.2805
triangulate:  [[[2, 0, 0, 1], [0, 1, 0, 1], [0, 0, 1, 1], [1, 1, 0, 1]], [[2, 0, 0, 1], [0, 1, 0, 1], [0, 0, 1, 1], [1, 0, 0, 1]], [[2, 0, 0, 1], [0, 0, 1, 1], [1, 1, 0, 1], [1, 0, 1, 1]], [[0, 0, 0, 1], [0, 1, 0, 1], [0, 0, 1, 1], [1, 0, 0, 1]]]
1/((b1 + b2 + 2*b3 - 8)*(b1 + b2 + b3 - 8)*(b2 + b3 - 4)*b2) + 1/((b1 + 2*b2 + 2*b3 - 8)*(b1 + b2 + 2*b3 - 8)*(b2 + b3 - 4)*b3) - 1/((b1 + 2*b2 + 2*b3 - 8)*(b1 + b2 + b3 - 4)*b2*b3) - 1/((b1 + b2 + b3 

In [9]:
b1 = var('b1')
b2 = var('b2')
b3 = var('b3')
series = -1/((b1 + 4*b2 + 4*b3 - 16)*(b1 + 3*b2 + 3*b3 - 12)*b2*b3) - 1/((b1 + 3*b2 + 3*b3 - 12)*(b1 + 2*b2 + 2*b3 - 8)*b2*b3) - 1/((b1 + 2*b2 + 2*b3 - 8)*(b1 + b2 + b3 - 4)*b2*b3) - 1/((b1 + b2 + b3 - 4)*b1*b2*b3)

vol = NSolve(series)
vol = round(vol, 5)
print vol

reset starting point:  -0.945155396429 0.0289554292775 0.868486906506
solution:  [ -7.92395209e+37   1.50674164e+36   8.29731517e+37]
reset starting point:  -0.24412925128 0.544078319647 0.905419694929
solution:  [ -3.79295174e+39   3.12744189e+38   3.13947208e+38]
reset starting point:  0.719115524516 0.623074859994 0.178613812636
solution:  [ 3.99992642  1.00000341  1.00000136]
Done.
0.25


In [28]:
b1 = var('b1')
b2 = var('b2')
b3 = var('b3')

Series = -1/((b1 + 4*b2 + 4*b3 - 16)*(b1 + 3*b2 + 3*b3 - 12)*b2*b3) - 1/((b1 + 3*b2 + 3*b3 - 12)*(b1 + 2*b2 + 2*b3 - 8)*b2*b3) - 1/((b1 + 2*b2 + 2*b3 - 8)*(b1 + b2 + b3 - 4)*b2*b3) - 1/((b1 + b2 + b3 - 4)*b1*b2*b3)

d1 = diff(Series, b1)
d2 = diff(Series, b2)
d3 = diff(Series, b3)
d = (d1, d2, d3)

#vol = fsolve(func, d, (0.1, 0.1, 0.1))
vol = fsolve(func, x0 = np.array([0.1, 0.1, 0.1]), args = d)

print vol
print Series(b1 = vol[0], b2 = vol[1], b3 = vol[2])


vol = solve([d1 == 0, d2 == 0, d3 == 0], b1, b2, b3)
vol = [vol[0][0].rhs(), vol[0][1].rhs(), vol[0][2].rhs()]

print vol
print Series(b1 = 4.0, b2 = 1.0, b3 = 1.0)
#print Series(b1 = vol[0][0].rhs(), b2 = vol[0][1].rhs(), b3 = vol[0][2].rhs())

[ 4.00006586  0.99999455  0.99999452]
0.2500000000509317
[4, 1, 1]


ValueError: power::eval(): division by zero

In [36]:
from sage.plot.point import Point
triangle = Polyhedron(vertices=[[2,0,0],[0,2,0],[0,0,2]])
ineqs = triangle.inequality_generator()
hrep = triangle.Hrepresentation()
#print hrep

test = vector(ZZ,[1,1,1])
#test = [0,0,0]

for ineq in hrep:
    print ineq
    print ineq.eval(test)

An equation (1, 1, 1) x - 2 == 0
1
An inequality (0, -1, -1) x + 2 >= 0
0
An inequality (0, 1, 0) x + 0 >= 0
1
An inequality (0, 0, 1) x + 0 >= 0
1


In [16]:
import numpy as np
np.random.uniform(low=-1, high=1)

-0.7208901274128736